## Table of Contents

1.  Passive Substrate: specify the reflectivity
3.  Passive Substrate: use a soil model
4.  Specify substrate for active simulations
4.  Include an atmosphere

Objective: Learn how to incorporate substrate into a snowpack and include an atmospheric component in the model run. Investigate the sensitivity to substrate and atmosphere parameters.

# Substrate

The substrate is an object itself, containing information on the reflectivity and other parameters if used to calculate reflectivity. The substrate object then forms part of the snowpack object.

## Passive: with reflectivity

The first passive example demonstrates how to add a substrate with known reflectivity. This example is for a near perfect absorber (emissivity = 0.98). The substrate is defined, then added to the snowpack.


In [ ]:
# Import statements
import numpy as np
from smrt import make_snowpack, make_model  # Core model functionality
from smrt.inputs.sensor_list import passive
from smrt.substrate.reflector import make_reflector
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Create substrate from known reflectivity
substrate = make_reflector(temperature=265, specular_reflection=0.02)

In [ ]:
# Make simple snowpack, including substrate
snow = make_snowpack([1], "exponential", temperature=[265],
                     density=[280], corr_length=[5e-5], substrate=substrate)

In [ ]:
# Or the another new way by using the + operator (works also with +=):

snow = make_snowpack([1], "exponential", temperature=[265],
                     density=[280], corr_length=[5e-5])
medium = snow + substrate

# Make model, specify sensor then run model
m = make_model("iba", "dort")
rad = passive(21e9, 55)
m.run(rad, medium).TbV()

### Challenges:

1. Plot a graph of how the brightness temperature varies with reflectivity
2. Compare a shallow snowpack with a deep snowpack

## Passive: with soil model

Alternatively the soil dielectric constant may be calculated from a theoretical model. The Wegmuller and Mätzler (1999) soil model is included in SMRT, other soil models may be contributed by taking a similar approach. This model, however, means that a soil dielectric constant model is required. Here we use the Dobson et al., (1985) model.


In [ ]:
from smrt import make_soil
soil = make_soil('soil_wegmuller', 'dobson85', temperature=265, roughness_rms=0.25, 
                 moisture=0.25, sand=0.01, clay=0.7, drymatter=1300)

### Challenges:

1. Make a snowpack with a soil substrate
2. How does this compare with the reflector case above?
2. What is the impact of roughness (or other soil parameters)?

## Active: prescribed backscatter

In [ ]:
# Define the substrate
from smrt.substrate.reflector_backscatter import make_reflector
reflector = make_reflector(temperature=265, specular_reflection=0., backscattering_coefficient={'VV': 0.1, 'HH': 0.1})

In [ ]:
# Need to define an active sensor
from smrt.inputs.sensor_list import active
from smrt.utils import dB
scatt = active(13e9, 45)

### Challenges:

1. Make a snowpack with a substrate
2. Run model and output results in dB
3. How does the output vary with backscattering coefficients?


# Atmosphere

Note: It is likely that how Atmosphere works in SMRT is going to change to deal with advanced atmopshere models. This may affect this part of the tutorial in the future.

Import module and define atmosphere. It is possible to include the downwelling atmospheric contribution (tbdown), and the upwelling contribution (tbup) and/or the atmospheric transmissivity (trans). The default values are tbup, tbdown = 0 and transmittivity = 1.

In [ ]:
from smrt.atmosphere.simple_isotropic_atmosphere import SimpleIsotropicAtmosphere
atmos = SimpleIsotropicAtmosphere(tbdown=30., tbup=6., trans=0.90)

### Challenges

1. Set up a snowpack / sensor and model configuration
2. Include atmosphere in model run (e.g. model.run(sensor, snowpack, atmosphere=atmos))
3. Investigate how output changes with atmospheric transmissivity